In [ ]:
import sys
!{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user >/dev/null

## Importing libraries

In [ ]:
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

import os
import json
import kfp
import string
import random
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeCode
from dkube.sdk.rsrcs import DkubeDataset
from dkube.sdk.rsrcs import DkubeModel

## Loading Components

In [ ]:
dkube_preprocessing_op = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
user = os.getenv('USERNAME')
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

api = DkubeApi(token=token)

## Create Dataset

In [ ]:
training_program = 'mm-demo'
dataset_name = 'mm-training-data'
model_name = 'mm-training-model'

In [ ]:
try:
    dataset = DkubeDataset(user, name=dataset_name)
    dataset.update_dataset_source(source='git')
    dataset.update_git_details(url = 'https://github.com/oneconvergence/dkube-examples-internal/blob/master/sklearn/insurance/program/training.csv')
    api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")
        

## Create Model

In [ ]:
try:
    model = DkubeModel(user, name=model_name)
    model.update_model_source(source='dvs')
    api.create_model(model)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")
    

## Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-insurance-pl',
    description='insurance-training-pl'
)
def insurance_pipeline(token):
    
    
    train       = dkube_training_op(container=json.dumps({"image": "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}),
                                    framework="sklearn", version="0.23.2",
                                    program=str(training_program), 
                                    run_script="python insurance/modeltraining/training.py",
                                    datasets=json.dumps([str(dataset_name)]), outputs=json.dumps([str(model_name)]),
                                    input_dataset_mounts='["/data/training-data"]',
                                    output_mounts='["/mm/model"]',
                                    auth_token=token)
    
    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": "ocdr/sklearnserver:0.23.2"}),
                                    transformer_image =json.dumps({"image": "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}),
                                    transformer_project=str(training_program),
                                    transformer_code='insurance/modeltraining/transformer.py', auth_token=token).after(train)


In [ ]:
client.create_run_from_pipeline_func(insurance_pipeline, arguments={'token':token})